In [6]:
import pandas as pd
import numpy as np
import optuna
import matplotlib.pyplot as plt
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.metrics import roc_auc_score
import warnings
import os
import plotly.io as pio
from optbinning import OptimalBinning


# Tắt cảnh báo
warnings.filterwarnings("ignore")

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [7]:
# # ============================================================
# # 1. LOAD DATA
# # ============================================================
# print(">>> [1/4] LOADING DATA...")

# # Đường dẫn (giữ nguyên của bạn)
# data_path = r'C:\Users\PC\Documents\GitHub\Khoa-luan\EBM'

# try:
#     X_train = pd.read_parquet(f'{data_path}/X_train.parquet')
#     y_train = pd.read_parquet(f'{data_path}/y_train.parquet').squeeze()

#     X_val = pd.read_parquet(f'{data_path}/X_oos.parquet')
#     y_val = pd.read_parquet(f'{data_path}/y_oos.parquet').squeeze()

#     X_test = pd.read_parquet(f'{data_path}/X_oot.parquet')
#     y_test = pd.read_parquet(f'{data_path}/y_oot.parquet').squeeze()
    
#     print(f"   Train: {X_train.shape} | Val: {X_val.shape} | Test: {X_test.shape}")

# except FileNotFoundError:
#     print("❌ LỖI: Không tìm thấy file Parquet.")
#     # Dừng chương trình tại đây nếu lỗi (trong Notebook thì dùng raise)
#     raise

# # ============================================================
# # 2. CONFIG FEATURE LISTS
# # ============================================================
# # List biến Numerical
# cols_num = [
#     'BASE_AUM', 'TUOI', 'INCOME', 'CBAL', 'AFLIMT_AVG', 'LTV', 
#     'N_AVG_DEPOSIT_12M', 'UTILIZATION_RATE', 'AMT_CASH_ADVANCE_12M',
#     'PCT_PAYMENT_TO_BALANCE', 'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 
#     'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'CBAL_SHORTTERM_LOAN', 
#     'CBAL_LONGTERM_LOAN', 'RATE_AVG', 'MAX_DPD_12M', 'AVG_OD_DPD_12M', 
#     'N_AVG_OVERDUE_CBAL_12M', 'CNT_CREDIT_CARDS', 'CNT_MIN_PAY_6M', 
#     'CNT_OTHER_PRODUCTS', 'CNT_DPD_30PLUS_6M', 'MOB', 'DURATION_MAX', 
#     'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX', 'MAX_NHOMNOCIC'
# ]

# # List biến Categorical
# cols_cat = [
#     'C_GIOITINH', 'TTHONNHAN', 'SEGMENT', 
#     'OCCUPATION_TYPE', 'PURCOD_MAX',      
#     'NHANVIENBIDV', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT', 'HAS_LONGTERM_LOAN'
# ]

# # Các cột cần loại bỏ (ID, Target nếu có trong X, v.v.)
# # Lưu ý: X_train thường không có Target, nhưng cứ để đây cho an toàn
# exclude_cols = ["SAMPLE_TYPE", "SOCIF", "REF_MONTH", "BAD_FLAG", "BAD_NEXT_12M"] 

# # Lấy danh sách feature cuối cùng có trong X_train
# final_features = [c for c in X_train.columns if c not in exclude_cols]

# print(f">>> [2/4] PREPROCESSING {len(final_features)} FEATURES...")
# feature_types_list = []
# missing_cols = []
# X_train_ebm = X_train[final_features].copy()
# # ============================================================
# # 3. ÉP KIỂU DỮ LIỆU (QUAN TRỌNG)
# # ============================================================
# # Lưu ý: Phải dùng đúng tên biến X_train, X_val, X_test thay vì train/oos/oot

# # ============================================================
# # 4. CHUẨN HÓA DỮ LIỆU & TRAIN (PHƯƠNG PHÁP PANDAS CATEGORY)
# # ============================================================
# print(">>> [3/4] PREPARING DATA WITH PANDAS CATEGORY DTYPE...")

# # 1. Tạo bản sao để xử lý
# X_train_ebm = X_train[final_features].copy()

# # 2. Xử lý ép kiểu dữ liệu
# # Thay vì dùng feature_types list, ta ép thẳng vào dtype của DataFrame
# for col in final_features:
#     # --- XỬ LÝ BIẾN CATEGORICAL ---
#     if col in cols_cat:
#         # Bước 1: FillNA -> String
#         # Bước 2: Ép sang 'category' (Đây là chìa khóa để EBM tự hiểu)
#         X_train_ebm[col] = X_train_ebm[col].fillna("MISSING").astype(str).astype('category')
        
#     # --- XỬ LÝ BIẾN NUMERICAL ---
#     elif col in cols_num:
#         X_train_ebm[col] = pd.to_numeric(X_train_ebm[col], errors='coerce')
        
#     # --- XỬ LÝ BIẾN SÓT (Mặc định đưa về Num) ---
#     else:
#         print(f"⚠️ Cảnh báo: Cột '{col}' chưa được phân loại -> Ép về số.")
#         X_train_ebm[col] = pd.to_numeric(X_train_ebm[col], errors='coerce')

# print("✅ Đã chuẩn hóa dtype: Các biến Cate đã chuyển thành 'category', Num thành 'float/int'.")

# # ============================================================
# # 5. TRAIN EBM
# # ============================================================
# print(">>> [4/4] TRAINING EBM...")

# # LƯU Ý QUAN TRỌNG: 
# # Bỏ tham số 'feature_types' đi. 
# # EBM sẽ tự đọc dtype 'category' của Pandas và xử lý đúng như ý bạn.
# ebm = ExplainableBoostingClassifier(
#     random_state=42,
#     interactions=0, 
#     n_jobs=-1,
# )

# # Train với dữ liệu đã ép kiểu category
# ebm.fit(X_train_ebm, y_train)

# print("✅ TRAINING XONG! KHÔNG CÒN LỖI.")

# # Show Dashboard
# ebm_global = ebm.explain_global()
# show(ebm_global)

In [8]:
# # 1. Tạo thư mục chứa ảnh
# output_folder = "EBM_Autoscale_Plots"
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Lấy danh sách tên biến
# features = ebm_global.feature_names
# print(f">>> Bắt đầu xuất {len(features)} biểu đồ (Chế độ Autoscale)...")

# # 2. Vòng lặp xuất ảnh
# for i, feature_name in enumerate(features):
#     try:
#         # Lấy Figure gốc từ EBM
#         fig = ebm_global.visualize(i)
        
#         # --- BƯỚC QUAN TRỌNG NHẤT: BẬT AUTOSCALE ---
#         # Lệnh này tương đương với việc bạn bấm nút "Autoscale" trên giao diện
#         fig.update_yaxes(autorange=True)
        
#         # Tùy chỉnh thêm (Title, Size) cho đẹp
#         fig.update_layout(
#             title=f"EBM Plot (Autoscaled): {feature_name}",
#             width=1200, height=600,
#             font=dict(size=14),
#             # Thêm dòng này để trục Y hiện rõ lưới grid cho dễ nhìn
#             yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey')
#         )
        
#         # Lưu file
#         safe_name = feature_name.replace("/", "_").replace(" ", "_")
#         file_path = f"{output_folder}/{i}_{safe_name}.png"
        
#         fig.write_image(file_path, scale=2)
#         print(f"   [OK] {feature_name}")
        
#     except Exception as e:
#         print(f"   [LỖI] Biến {feature_name}: {e}")

# print(f"\n✅ Xong! Mở thư mục '{output_folder}' để xem thành quả.")

In [9]:
# increasing = ['CBAL', 'LTV', 'ULTILIZATION_RATE', 'CNT_CREDIT_CARDS', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M', 'DTI_RATIO', 'MOB', 'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'DURATION_MAX', 'REMAINING_DURATION_MAX', 'MAX_DPD_12M', 'MAC_NHOMNOCIC', 'N_AVG__OVERDUE_CBAL_12M']
# decreasing = ['BASE_AUM', 'TUOI', 'INCOME', 'AFLIMT_AVG', 'N_AVG_DEPOSIT_12M', 'AMT_CASH_ADVANCE_12M', 'AVG_DAYS_PAST_DUE', 'CNT_OTHER_PRODUCTS', 'CBAL_SHORTTERM_LOAN', "CBAL_LONGTERM_LOAN", 'CNT_DPD_30PLUS_6M', 'TIME_TO_OP_MAX', 'RATE_AVG']


In [10]:
# ============================================================
# 1. LOAD DATA
# ============================================================
print(">>> [1/4] LOADING DATA...")

def calculate_gini(model, X, y, label=""):
    prob = model.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, prob)
    gini = 2 * auc - 1
    if label:
        print(f"   [{label}] AUC: {auc:.4f} | GINI: {gini:.4f}")
    return gini

# Đường dẫn (giữ nguyên của bạn)
data_path = r'C:\Users\PC\Documents\GitHub\Khoa-luan\EBM'

try:
    X_train = pd.read_parquet(f'{data_path}/X_train.parquet')
    y_train = pd.read_parquet(f'{data_path}/y_train.parquet').squeeze()

    X_val = pd.read_parquet(f'{data_path}/X_oos.parquet')
    y_val = pd.read_parquet(f'{data_path}/y_oos.parquet').squeeze()

    X_test = pd.read_parquet(f'{data_path}/X_oot.parquet')
    y_test = pd.read_parquet(f'{data_path}/y_oot.parquet').squeeze()
    
    print(f"   Train: {X_train.shape} | Val: {X_val.shape} | Test: {X_test.shape}")

except FileNotFoundError:
    print("❌ LỖI: Không tìm thấy file Parquet.")
    # Dừng chương trình tại đây nếu lỗi (trong Notebook thì dùng raise)
    raise

# ============================================================
# 2. CONFIG FEATURE LISTS
# ============================================================
# List biến Numerical
cols_num = [
    'BASE_AUM', 'TUOI', 'INCOME', 'CBAL', 'AFLIMT_AVG', 'LTV', 
    'N_AVG_DEPOSIT_12M', 'UTILIZATION_RATE', 'AMT_CASH_ADVANCE_12M',
    'PCT_PAYMENT_TO_BALANCE', 'AVG_DAYS_PAST_DUE', 'DTI_RATIO', 
    'LIMIT_TO_INCOME', 'AMT_VAR_6M', 'CBAL_SHORTTERM_LOAN', 
    'CBAL_LONGTERM_LOAN', 'RATE_AVG', 'MAX_DPD_12M', 'AVG_OD_DPD_12M', 
    'N_AVG_OVERDUE_CBAL_12M', 'CNT_CREDIT_CARDS', 'CNT_MIN_PAY_6M', 
    'CNT_OTHER_PRODUCTS', 'CNT_DPD_30PLUS_6M', 'MOB', 'DURATION_MAX', 
    'REMAINING_DURATION_MAX', 'TIME_TO_OP_MAX', 'MAX_NHOMNOCIC'
]

# List biến Categorical
cols_cat = [
    'C_GIOITINH', 'TTHONNHAN', 'SEGMENT', 
    'OCCUPATION_TYPE', 'PURCOD_MAX',      
    'NHANVIENBIDV', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT', 'HAS_LONGTERM_LOAN'
]

# Các cột cần loại bỏ (ID, Target nếu có trong X, v.v.)
# Lưu ý: X_train thường không có Target, nhưng cứ để đây cho an toàn
exclude_cols = ["SAMPLE_TYPE", "SOCIF", "REF_MONTH", "BAD_FLAG", "BAD_NEXT_12M"] 

# Lấy danh sách feature cuối cùng có trong X_train
final_features = [c for c in X_train.columns if c not in exclude_cols]
print(f">>> [2/4] PREPROCESSING {len(final_features)} FEATURES...")
feature_types_list = []
missing_cols = []
X_train_ebm = X_train[final_features].copy()
# ============================================================
# 3. ÉP KIỂU DỮ LIỆU (QUAN TRỌNG)
# ============================================================
# Lưu ý: Phải dùng đúng tên biến X_train, X_val, X_test thay vì train/oos/oot

# ============================================================
# 4. CHUẨN HÓA DỮ LIỆU & TRAIN (PHƯƠNG PHÁP PANDAS CATEGORY)
# ============================================================
print(">>> [3/4] PREPARING DATA WITH PANDAS CATEGORY DTYPE...")

# 1. Tạo bản sao để xử lý
X_train_ebm = X_train[final_features].copy()

# 2. Xử lý ép kiểu dữ liệu
# Thay vì dùng feature_types list, ta ép thẳng vào dtype của DataFrame
for col in final_features:
    # --- XỬ LÝ BIẾN CATEGORICAL ---
    if col in cols_cat:
        # Bước 1: FillNA -> String
        # Bước 2: Ép sang 'category' (Đây là chìa khóa để EBM tự hiểu)
        X_train_ebm[col] = X_train_ebm[col].fillna("MISSING").astype(str).astype('category')
        
    # --- XỬ LÝ BIẾN NUMERICAL ---
    elif col in cols_num:
        X_train_ebm[col] = pd.to_numeric(X_train_ebm[col], errors='coerce')
        
    # --- XỬ LÝ BIẾN SÓT (Mặc định đưa về Num) ---
    else:
        print(f"⚠️ Cảnh báo: Cột '{col}' chưa được phân loại -> Ép về số.")
        X_train_ebm[col] = pd.to_numeric(X_train_ebm[col], errors='coerce')

print("✅ Đã chuẩn hóa dtype: Các biến Cate đã chuyển thành 'category', Num thành 'float/int'.")

# ============================================================
# 5. TRAIN EBM
# ============================================================
print(">>> [4/4] TRAINING EBM...")
increasing_cols = [
    'CBAL', 'LTV', 'UTILIZATION_RATE', # Đã sửa ULTILIZATION -> UTILIZATION
    'CNT_CREDIT_CARDS', 'PCT_PAYMENT_TO_BALANCE', 'CNT_MIN_PAY_6M', 
    'DTI_RATIO', 'MOB', 'LIMIT_TO_INCOME', 'AMT_VAR_6M', 
    'DURATION_MAX', 'REMAINING_DURATION_MAX', 'MAX_DPD_12M', 
    'MAX_NHOMNOCIC', # Đã sửa MAC -> MAX
    'N_AVG_OVERDUE_CBAL_12M' # Đã sửa N_AVG__ -> N_AVG_ (bỏ 1 dấu gạch)
]

decreasing_cols = [
    'BASE_AUM', 'TUOI', 'INCOME', 'AFLIMT_AVG', 'N_AVG_DEPOSIT_12M', 
    'AMT_CASH_ADVANCE_12M', 'AVG_DAYS_PAST_DUE', 'CNT_OTHER_PRODUCTS', 
    'CBAL_SHORTTERM_LOAN', "CBAL_LONGTERM_LOAN", 'CNT_DPD_30PLUS_6M', 
    'TIME_TO_OP_MAX', 'RATE_AVG'
]

# 2. Map vào Dictionary
constraints_list = []

# Gán +1 cho nhóm Increasing
for col in X_train_ebm.columns:
    # Nếu là biến Categorical (đã ép kiểu category) -> Bắt buộc là 0
    # (Kiểm tra xem cột có phải dạng category không)
    if pd.api.types.is_categorical_dtype(X_train_ebm[col]):
        constraints_list.append(0)
        
    # Nếu là biến số cần ép Tăng (+1)
    elif col in increasing_cols:
        constraints_list.append(1)
        
    # Nếu là biến số cần ép Giảm (-1)
    elif col in decreasing_cols:
        constraints_list.append(-1)
    

# LƯU Ý QUAN TRỌNG: 
# Bỏ tham số 'feature_types' đi. 
# EBM sẽ tự đọc dtype 'category' của Pandas và xử lý đúng như ý bạn.
ebm = ExplainableBoostingClassifier(
    random_state=42,
    monotone_constraints=constraints_list,
    interactions=0, 
    n_jobs=-1,
)

# Train với dữ liệu đã ép kiểu category
ebm.fit(X_train_ebm, y_train)

print("   Kết quả Baseline:")
gini_train = calculate_gini(ebm, X_train, y_train, "TRAIN-Base")
gini_base_oos = calculate_gini(ebm, X_val, y_val, "OOS-Base")
gini_base_oot = calculate_gini(ebm, X_test, y_test, "OOT-Base")

# Show Dashboard
ebm_global = ebm.explain_global()
show(ebm_global)

>>> [1/4] LOADING DATA...
   Train: (1137807, 28) | Val: (300317, 28) | Test: (302113, 28)
>>> [2/4] PREPROCESSING 28 FEATURES...
>>> [3/4] PREPARING DATA WITH PANDAS CATEGORY DTYPE...
✅ Đã chuẩn hóa dtype: Các biến Cate đã chuyển thành 'category', Num thành 'float/int'.
>>> [4/4] TRAINING EBM...
   Kết quả Baseline:
   [TRAIN-Base] AUC: 0.8716 | GINI: 0.7433
   [OOS-Base] AUC: 0.8786 | GINI: 0.7573
   [OOT-Base] AUC: 0.8775 | GINI: 0.7551


<!-- http://127.0.0.1:7001/2300040830576/ -->

In [11]:
# # 1. Tạo thư mục chứa ảnh
# output_folder = "EBM_Autoscale_Plots_mono"
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Lấy danh sách tên biến
# features = ebm_global.feature_names
# print(f">>> Bắt đầu xuất {len(features)} biểu đồ (Chế độ Autoscale)...")

# # 2. Vòng lặp xuất ảnh
# for i, feature_name in enumerate(features):
#     try:
#         # Lấy Figure gốc từ EBM
#         fig = ebm_global.visualize(i)
        
#         # --- BƯỚC QUAN TRỌNG NHẤT: BẬT AUTOSCALE ---
#         # Lệnh này tương đương với việc bạn bấm nút "Autoscale" trên giao diện
#         fig.update_yaxes(autorange=True)
        
#         # Tùy chỉnh thêm (Title, Size) cho đẹp
#         fig.update_layout(
#             title=f"EBM Plot (Autoscaled): {feature_name}",
#             width=1200, height=600,
#             font=dict(size=14),
#             # Thêm dòng này để trục Y hiện rõ lưới grid cho dễ nhìn
#             yaxis=dict(showgrid=True, gridwidth=1, gridcolor='LightGrey')
#         )
        
#         # Lưu file
#         safe_name = feature_name.replace("/", "_").replace(" ", "_")
#         file_path = f"{output_folder}/{i}_{safe_name}.png"
        
#         fig.write_image(file_path, scale=2)
#         print(f"   [OK] {feature_name}")
        
#     except Exception as e:
#         print(f"   [LỖI] Biến {feature_name}: {e}")

# print(f"\n✅ Xong! Mở thư mục '{output_folder}' để xem thành quả.")

In [12]:
# Lấy tên biến và độ quan trọng tương ứng
feature_names = ebm.feature_names_in_
importances = ebm.term_importances()

# Tạo dataframe để dễ nhìn
df_imp = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sắp xếp từ cao xuống thấp
df_imp = df_imp.sort_values(by='Importance', ascending=False)

# In ra toàn bộ danh sách
print(f"Tổng số biến trong mô hình: {len(df_imp)}")
print(df_imp)

Tổng số biến trong mô hình: 28
                   Feature  Importance
26             MAX_DPD_12M    1.040961
5                      LTV    0.643910
27  N_AVG_OVERDUE_CBAL_12M    0.301791
14               DTI_RATIO    0.235779
6        N_AVG_DEPOSIT_12M    0.201965
7          FLAG_SALARY_ACC    0.184082
4                   INCOME    0.168187
3                     TUOI    0.143377
19     CBAL_SHORTTERM_LOAN    0.063732
12          CNT_MIN_PAY_6M    0.060575
20      CBAL_LONGTERM_LOAN    0.058966
8             FLAG_DEPOSIT    0.038714
17         LIMIT_TO_INCOME    0.037543
24                RATE_AVG    0.027796
13       AVG_DAYS_PAST_DUE    0.015640
11  PCT_PAYMENT_TO_BALANCE    0.005984
15                     MOB    0.005569
18              AMT_VAR_6M    0.005389
25              PURCOD_MAX    0.004451
0               C_GIOITINH    0.004421
21       CNT_DPD_30PLUS_6M    0.004295
1                TTHONNHAN    0.003832
22         OCCUPATION_TYPE    0.003148
23            DURATION_MAX    0.0

In [14]:
# ============================================================
# 1. CHỐT DANH SÁCH 19 BIẾN QUAN TRỌNG
# ============================================================
print(">>> [1/4] FILTERING FEATURES...")

# Danh sách 19 biến "Golden Features"
selected_features = [
    # --- Nhóm Rủi ro Cao (+1) ---
    'LTV', 'MAX_DPD_12M', 'N_AVG_OVERDUE_CBAL_12M',
    'DTI_RATIO', 'CNT_MIN_PAY_6M', 
    'LIMIT_TO_INCOME',
    
    # --- Nhóm Tốt/Tài sản (-1) ---
     'INCOME', 'N_AVG_DEPOSIT_12M', 
    'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'RATE_AVG',
    
    # --- Nhóm Khác (0) ---
    'TUOI', 'FLAG_SALARY_ACC', 'FLAG_DEPOSIT'
]

# Định nghĩa nhóm Cate và Num
cols_cat = ['FLAG_SALARY_ACC', 'FLAG_DEPOSIT']
cols_num = [c for c in selected_features if c not in cols_cat]

# ============================================================
# 2. CHUẨN HÓA DỮ LIỆU (TRAIN, VAL, TEST)
# ============================================================
print(">>> [2/4] PREPARING DATA TYPES...")

# Tạo bản sao gọn nhẹ chỉ chứa 19 cột
X_train_ebm = X_train[selected_features].copy()
X_val_ebm   = X_val[selected_features].copy()
X_test_ebm  = X_test[selected_features].copy()

# Hàm ép kiểu thống nhất
def convert_types(df):
    for col in selected_features:
        if col in cols_cat:
            # Ép về category để EBM tự hiểu
            df[col] = df[col].fillna("MISSING").astype(str).astype('category')
        else:
            # Ép về số
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# Áp dụng cho cả 3 tập (để tính Gini OOS/OOT không bị lỗi)
X_train_ebm = convert_types(X_train_ebm)
X_val_ebm   = convert_types(X_val_ebm)
X_test_ebm  = convert_types(X_test_ebm)

print("✅ Đã ép kiểu xong (Category & Numeric).")

# ============================================================
# 3. CẤU HÌNH ÉP CHIỀU (MONOTONICITY)
# ============================================================
print(">>> [3/4] CONFIGURING CONSTRAINTS...")

# Nhóm Tăng (+1): Giá trị càng cao -> Rủi ro càng cao
increasing_cols = [
    'LTV', 'MAX_NHOMNOCIC', 'MAX_DPD_12M', 'N_AVG_OVERDUE_CBAL_12M',
    'DTI_RATIO', 'UTILIZATION_RATE', 'CBAL', 'CNT_MIN_PAY_6M', 
    'LIMIT_TO_INCOME' 
]

# Nhóm Giảm (-1): Giá trị càng cao -> Rủi ro càng thấp (Tốt)
decreasing_cols = [
    'BASE_AUM', 'INCOME', 'AFLIMT_AVG', 'N_AVG_DEPOSIT_12M',
    'CBAL_SHORTTERM_LOAN', 'CBAL_LONGTERM_LOAN', 'RATE_AVG'
]

# Tạo list constraints khớp 100% với thứ tự cột trong X_train_ebm
constraints_list = []
for col in X_train_ebm.columns:
    if pd.api.types.is_categorical_dtype(X_train_ebm[col]):
        constraints_list.append(0)   # Biến Cate -> 0
    elif col in increasing_cols:
        constraints_list.append(1)   # Ép tăng
    elif col in decreasing_cols:
        constraints_list.append(-1)  # Ép giảm
    else:
        constraints_list.append(0)   # Còn lại (TUOI) -> 0

# ============================================================
# 4. TRAIN & EVALUATE
# ============================================================
print(">>> [4/4] TRAINING EBM (19 VARIABLES)...")

ebm = ExplainableBoostingClassifier(
    random_state=42,
    monotone_constraints=constraints_list, # List chuẩn
    interactions=0, 
    n_jobs=-1,
    outer_bags=8,
    inner_bags=0
)

# Train
ebm.fit(X_train_ebm, y_train)
print("✅ TRAINING THÀNH CÔNG!")

# Tính Gini
print("\n---------------- KẾT QUẢ ----------------")
calculate_gini(ebm, X_train_ebm, y_train, "TRAIN")
calculate_gini(ebm, X_val_ebm, y_val, "OOS")
calculate_gini(ebm, X_test_ebm, y_test, "OOT")

# Show Dashboard
ebm_global = ebm.explain_global()
show(ebm_global)

>>> [1/4] FILTERING FEATURES...
>>> [2/4] PREPARING DATA TYPES...
✅ Đã ép kiểu xong (Category & Numeric).
>>> [3/4] CONFIGURING CONSTRAINTS...
>>> [4/4] TRAINING EBM (19 VARIABLES)...
✅ TRAINING THÀNH CÔNG!

---------------- KẾT QUẢ ----------------
   [TRAIN (19 Vars)] AUC: 0.8716 | GINI: 0.7433
   [OOS (19 Vars)] AUC: 0.8788 | GINI: 0.7576
   [OOT (19 Vars)] AUC: 0.8776 | GINI: 0.7552


<!-- http://127.0.0.1:7001/2300039833360/ -->

In [15]:
print("\n>>> [4/6] CREATE MASTER PD-DATAFRAME...")

# Predict PD
pd_train = ebm.predict_proba(X_train)[:, 1]
pd_oos   = ebm.predict_proba(X_val)[:, 1]
pd_oot   = ebm.predict_proba(X_test)[:, 1]

# Tạo DF cho từng tập
df_train = pd.DataFrame({
    "DATA_TYPE": "TRAIN",
    "y": y_train,
    "PD": pd_train
})

df_oos = pd.DataFrame({
    "DATA_TYPE": "OOS",
    "y": y_val,
    "PD": pd_oos
})

df_oot = pd.DataFrame({
    "DATA_TYPE": "OOT",
    "y": y_test,
    "PD": pd_oot
})

# Gộp lại
df = pd.concat([df_train, df_oos, df_oot], ignore_index=True)

print("→ DF created:", df.shape)
print(df.head())


>>> [4/6] CREATE MASTER PD-DATAFRAME...
→ DF created: (1740237, 3)
  DATA_TYPE  y        PD
0     TRAIN  0  0.015252
1     TRAIN  1  0.200559
2     TRAIN  0  0.128738
3     TRAIN  0  0.118896
4     TRAIN  0  0.099548


In [16]:
print("\n>>> [5/6] CONVERT PD → SCORE")

PDO = 20
ODDS0 = 50
SCORE0 = 600

Factor = PDO / np.log(2)
Offset = SCORE0 + Factor * np.log(ODDS0)

def pd_to_score(pd):
    pd = np.clip(pd, 1e-6, 1 - 1e-6)
    log_odds = np.log(pd / (1 - pd))
    return Offset - Factor * log_odds

df["SCORE"] = pd_to_score(df["PD"])
print(df.head())



>>> [5/6] CONVERT PD → SCORE
  DATA_TYPE  y        PD       SCORE
0     TRAIN  0  0.015252  833.131738
1     TRAIN  1  0.200559  752.776352
2     TRAIN  0  0.128738  768.050556
3     TRAIN  0  0.118896  770.669436
4     TRAIN  0  0.099548  776.420916


In [17]:
print("\n>>> [6/6] FIT OPT BINNING ON TRAIN...")

optb = OptimalBinning(
    name="rating_scale",
    dtype="numerical",
    monotonic_trend="descending",  # Risk cao → Score thấp
    max_n_bins=10
)

optb.fit(df[df["DATA_TYPE"]=="TRAIN"]["SCORE"], df[df["DATA_TYPE"]=="TRAIN"]["y"])
df["BIN"] = optb.transform(df["SCORE"])



>>> [6/6] FIT OPT BINNING ON TRAIN...


In [18]:
rating_labels = ["AAA","AA+","AA","AA-","A+","A","A-","BBB","BB","B"]

# Trung bình PD từng BIN → để xếp thứ tự hạng
bin_mean_pd = df.groupby("BIN")["PD"].mean().sort_values()
sorted_bins = bin_mean_pd.index.tolist()

rating_map = {b: rating_labels[i] for i, b in enumerate(sorted_bins)}

df["RATING"] = df["BIN"].map(rating_map)

print("RATING DONE")
print(df.head())

RATING DONE
  DATA_TYPE  y        PD       SCORE       BIN RATING
0     TRAIN  0  0.015252  833.131738  1.607355    AA-
1     TRAIN  1  0.200559  752.776352 -1.030027    BBB
2     TRAIN  0  0.128738  768.050556 -0.591242     A-
3     TRAIN  0  0.118896  770.669436 -0.591242     A-
4     TRAIN  0  0.099548  776.420916 -0.591242     A-


In [19]:
# --- TIẾP NỐI TỪ CODE CỦA BẠN ---

# 1. Định nghĩa thứ tự hạng (để sắp xếp bảng cho đúng chuẩn từ Tốt -> Xấu)
rating_order = ["AAA", "AA+", "AA", "AA-", "A+", "A", "A-", "BBB", "BB", "B"]

# 2. Groupby để tính toán các chỉ số
# Lưu ý: Cột target của bạn trong df tên là "y"
summary = df.groupby("RATING").agg({
    "SCORE": ["min", "max"],      # Để lấy khoảng điểm
    "PD": ["mean", "max"],        # Mean PD để so sánh, Max PD để tìm điểm cắt (Cutpoint)
    "y": ["count", "mean"]        # Count = Số lượng, Mean = Default Rate thực tế
}).reset_index()

# Làm phẳng MultiIndex của cột sau khi group
summary.columns = ['RATING', 'MIN_SCORE', 'MAX_SCORE', 'MEAN_PD', 'MAX_PD_BIN', 'COUNT', 'DEFAULT_RATE']

# 3. Sắp xếp bảng theo thứ tự Rating (AAA -> B)
summary['RATING'] = pd.Categorical(summary['RATING'], categories=rating_order, ordered=True)
summary = summary.sort_values('RATING').reset_index(drop=True)

# 4. Tính toán các cột hiển thị (Format lại cho đẹp)
# Tính % tỷ trọng
total_obs = summary['COUNT'].sum()
summary['PCT'] = summary['COUNT'] / total_obs

# Tạo cột Score Range "Min -> Max"
# Lưu ý: Vì Score và PD ngược chiều (Score cao = PD thấp), 
# nên hạng AAA sẽ có Score cao nhất. Ta hiển thị từ Min -> Max cho thuận mắt.
summary['SCORE_RANGE'] = summary.apply(lambda x: f"{x['MIN_SCORE']:.2f} → {x['MAX_SCORE']:.2f}", axis=1)

# 5. Trích xuất Cutpoints (Điểm cắt PD)
# Điểm cắt chính là PD lớn nhất (Max PD) của từng hạng.
# Khách hàng có PD <= Cutpoint của AAA sẽ vào AAA.
# Ta lấy values[:-1] để bỏ qua điểm cắt cuối cùng (của hạng B, thường là 1.0)
cutpoints = summary['MAX_PD_BIN'].values[:-1]

# 6. Sắp xếp lại cột để in ra bảng Final (giống mẫu bạn yêu cầu)
final_table = summary[['RATING', 'SCORE_RANGE', 'COUNT', 'PCT', 'MEAN_PD', 'DEFAULT_RATE']]

# --- IN KẾT QUẢ ---
print("\n>>> CUTPOINTS (PD Thresholds):")
print(cutpoints)

print("\n>>> RATING SUMMARY TABLE:")
# Format hiển thị dạng bảng đẹp
pd.set_option('display.float_format', '{:.6f}'.format) # Để nhìn rõ số PD nhỏ
print(final_table.to_string(index=False, formatters={
    'PCT': '{:.2%}'.format,         # Dạng % (vd: 8.42%)
    'MEAN_PD': '{:.6f}'.format,     # Giữ nguyên số thập phân hoặc {:.2%} tùy bạn
    'DEFAULT_RATE': '{:.6f}'.format 
}))


>>> CUTPOINTS (PD Thresholds):
[0.00134957 0.00393775 0.00944629 0.02339609 0.05875492 0.09367731
 0.14628037 0.22327075 0.32770728]

>>> RATING SUMMARY TABLE:
RATING     SCORE_RANGE  COUNT    PCT  MEAN_PD DEFAULT_RATE
   AAA 903.50 → 953.29 121246  6.97% 0.000901     0.000792
   AA+ 872.53 → 903.50 221304 12.72% 0.002580     0.002883
    AA 847.12 → 872.53 303698 17.45% 0.006296     0.006421
   AA- 820.55 → 847.12 315623 18.14% 0.015484     0.016054
    A+ 792.91 → 820.55 265124 15.23% 0.036965     0.035994
     A 778.36 → 792.91 104977  6.03% 0.075155     0.081018
    A- 763.78 → 778.36 120787  6.94% 0.118950     0.129029
   BBB 748.85 → 763.78 121921  7.01% 0.181447     0.183947
    BB 733.61 → 748.85  86759  4.99% 0.268824     0.254925
     B 649.05 → 733.61  78798  4.53% 0.460983     0.455595


In [20]:
import pandas as pd

# ============================================================
# EXPORT EBM SCORECARD TO EXCEL
# ============================================================
print(">>> EXTRACTING SCORECARD...")

scorecard_data = []

# 1. Lấy điểm cơ sở (Intercept)
# Đây là điểm khởi đầu cho mọi khách hàng
intercept = ebm.intercept_[0] if hasattr(ebm, "intercept_") else ebm.intercept_
scorecard_data.append({
    "Feature": "INTERCEPT (Base Score)",
    "Type": "Base",
    "Bin_Index": -1,
    "Range/Value": "All",
    "Score_Raw": intercept,
    "Score_Points": intercept * (20 / np.log(2)) # Scale sang điểm hồ sơ nếu cần
})

# 2. Duyệt qua từng biến để lấy bảng điểm
# ebm.term_names_ : Tên biến
# ebm.term_scores_: Điểm số tương ứng từng bin
# ebm.bins_       : Các điểm cắt (Cutoffs) của bin
for i, feature_name in enumerate(ebm.term_names_):
    
    # Lấy thông tin của biến thứ i
    # Lưu ý: EBM lưu scores và bins theo index
    scores = ebm.term_scores_[i]
    
    # Xử lý biến Continuous (Số) vs Categorical (Chữ)
    feature_type = ebm.feature_types_in_[i]
    
    if feature_type == 'continuous':
        # Với biến số, ta có các điểm cắt (cutoffs)
        # bins[i] chứa các ngưỡng cắt. Ví dụ: [20, 30, 40]
        # -> Các khoảng là: (-inf, 20], (20, 30], (30, 40], (40, +inf)
        # Số lượng scores luôn nhiều hơn số lượng cutoffs 1 đơn vị
        cutoffs = ebm.bins_[i][0] 
        
        # Bin đầu tiên (-inf -> cutoff 0)
        scorecard_data.append({
            "Feature": feature_name,
            "Type": "Continuous",
            "Bin_Index": 0,
            "Range/Value": f"< {cutoffs[0]:.4f}",
            "Score_Raw": scores[0]
        })
        
        # Các bin ở giữa
        for k in range(1, len(cutoffs)):
            scorecard_data.append({
                "Feature": feature_name,
                "Type": "Continuous",
                "Bin_Index": k,
                "Range/Value": f"{cutoffs[k-1]:.4f} - {cutoffs[k]:.4f}",
                "Score_Raw": scores[k]
            })
            
        # Bin cuối cùng (> cutoff cuối)
        scorecard_data.append({
            "Feature": feature_name,
            "Type": "Continuous",
            "Bin_Index": len(cutoffs),
            "Range/Value": f">= {cutoffs[-1]:.4f}",
            "Score_Raw": scores[-1]
        })
        
    elif feature_type == 'nominal': # Categorical
        # Với biến Cate, bins[i][0] chứa danh sách các giá trị (labels)
        labels = ebm.bins_[i][0]
        
        # EBM mapping từng label vào score
        # Lưu ý: Có thể có bin "Unseen/Missing" tùy version, nhưng thường khớp index
        for k, label in enumerate(labels):
             scorecard_data.append({
                "Feature": feature_name,
                "Type": "Categorical",
                "Bin_Index": k,
                "Range/Value": str(label),
                "Score_Raw": scores[k]
            })

# 3. Tạo DataFrame
df_scorecard = pd.DataFrame(scorecard_data)

# 4. Tính điểm quy đổi (Scaled Score)
# Công thức Score = Offset + Factor * (-Raw_Score) 
# (Lưu ý dấu: EBM ra Raw Score dương là Rủi ro cao -> Cần trừ đi)
# Ở đây ta tính "Contribution" (Điểm đóng góp): Points = -Raw * Factor
factor = 20 / np.log(2)
df_scorecard["Points"] = -df_scorecard["Score_Raw"] * factor
df_scorecard["Points"] = df_scorecard["Points"].round(0) # Làm tròn điểm

# 5. Xuất ra Excel
print(df_scorecard.head(10))
df_scorecard.to_excel("EBM_Scorecard_Matrix.xlsx", index=False)
print("✅ Đã xuất ma trận tính điểm ra file 'EBM_Scorecard_Matrix.xlsx'")

>>> EXTRACTING SCORECARD...
                  Feature        Type  Bin_Index      Range/Value  Score_Raw  Score_Points     Points
0  INTERCEPT (Base Score)        Base         -1              All  -3.760922   -108.517271 109.000000
1                     LTV  Continuous          0         < 2.7556   0.000000           NaN  -0.000000
2                     LTV  Continuous          1  2.7556 - 3.0518  -1.335303           NaN  39.000000
3                     LTV  Continuous          2  3.0518 - 3.2593  -1.335303           NaN  39.000000
4                     LTV  Continuous          3  3.2593 - 3.4235  -1.335303           NaN  39.000000
5                     LTV  Continuous          4  3.4235 - 3.5535  -1.335303           NaN  39.000000
6                     LTV  Continuous          5  3.5535 - 3.6802  -1.333843           NaN  38.000000
7                     LTV  Continuous          6  3.6802 - 3.7878  -1.333843           NaN  38.000000
8                     LTV  Continuous          7  3.78

In [21]:
from scipy.stats import ks_2samp
from sklearn.metrics import brier_score_loss

# ============================================================
# HÀM TÍNH TOÁN CÁC CHỈ SỐ NÂNG CAO
# ============================================================
def calculate_advanced_metrics(model, X, y, dataset_label=""):
    # 1. Dự báo xác suất
    y_prob = model.predict_proba(X)[:, 1]
    
    # 2. Tính KS Statistic
    # Tách xác suất của nhóm Good (0) và Bad (1)
    prob_good = y_prob[y == 0]
    prob_bad  = y_prob[y == 1]
    ks_stat, p_value = ks_2samp(prob_good, prob_bad)
    
    # 3. Tính Brier Score (Càng thấp càng tốt)
    brier = brier_score_loss(y, y_prob)
    
    # 4. Tính Bad Capture Rate @ Top 10% & 20%
    # Tạo DataFrame tạm để sort
    df_temp = pd.DataFrame({'y': y, 'prob': y_prob})
    df_temp = df_temp.sort_values(by='prob', ascending=False)
    
    total_bads = df_temp['y'].sum()
    n_rows = len(df_temp)
    
    # Top 10%
    top_10_idx = int(n_rows * 0.1)
    bads_in_top_10 = df_temp.iloc[:top_10_idx]['y'].sum()
    capture_rate_10 = bads_in_top_10 / total_bads if total_bads > 0 else 0
    
    # Top 20%
    top_20_idx = int(n_rows * 0.2)
    bads_in_top_20 = df_temp.iloc[:top_20_idx]['y'].sum()
    capture_rate_20 = bads_in_top_20 / total_bads if total_bads > 0 else 0
    
    print(f">>> REPORT FOR: {dataset_label}")
    print(f"   • KS Statistic:       {ks_stat:.4f} ({(ks_stat*100):.2f}%)")
    print(f"   • Brier Score:        {brier:.6f}")
    print(f"   • Capture Rate @ 10%: {capture_rate_10:.2%} (Tóm được bao nhiêu Bad trong top 10% rủi ro nhất)")
    print(f"   • Capture Rate @ 20%: {capture_rate_20:.2%}")
    print("-" * 60)
    
    return ks_stat, brier, capture_rate_10

# ============================================================
# HÀM TÍNH PSI (ĐỘ ỔN ĐỊNH)
# ============================================================
def calculate_psi(expected, actual, buckets=10):
    # expected: Phân phối PD/Score tập Train
    # actual: Phân phối PD/Score tập OOT
    
    def scale_range(input, min, max):
        input += -(np.min(input))
        input /= np.max(input) / (max - min)
        input += min
        return input

    breakpoints = np.arange(0, buckets + 1) / (buckets) * 100
    
    # Chia bin dựa trên tập Train (Expected)
    breakpoints = np.percentile(expected, breakpoints)
    
    # Tính % số lượng mẫu trong từng bin
    expected_percents = np.histogram(expected, breakpoints)[0] / len(expected)
    actual_percents = np.histogram(actual, breakpoints)[0] / len(actual)
    
    # Xử lý division by zero
    expected_percents = np.where(expected_percents == 0, 0.0001, expected_percents)
    actual_percents = np.where(actual_percents == 0, 0.0001, actual_percents)
    
    # Công thức PSI
    psi_value = np.sum((expected_percents - actual_percents) * np.log(expected_percents / actual_percents))
    
    return psi_value

# ============================================================
# CHẠY BÁO CÁO SO SÁNH
# ============================================================
print("\n" + "="*40)
print("📊 BÁO CÁO CHỈ SỐ NÂNG CAO CHO EBM")
print("="*40)

# 1. Tính các chỉ số KS, Brier, Capture Rate trên OOT
# (Chỉ số trên OOT là quan trọng nhất để so sánh công bằng)
calculate_advanced_metrics(ebm, X_test_ebm, y_test, "OOT DATASET (Tương lai)")

# 2. Tính PSI (Train vs OOT)
# Dùng xác suất dự báo (PD) để tính PSI
prob_train = ebm.predict_proba(X_train_ebm)[:, 1]
prob_oot   = ebm.predict_proba(X_test_ebm)[:, 1]

psi_score = calculate_psi(prob_train, prob_oot)

print(f">>> STABILITY (PSI - Train vs OOT): {psi_score:.4f}")
if psi_score < 0.1:
    print("   ✅ PSI < 0.1: Mô hình rất ổn định.")
elif psi_score < 0.25:
    print("   ⚠️ PSI 0.1-0.25: Mô hình biến động nhẹ.")
else:
    print("   ❌ PSI > 0.25: Mô hình không ổn định (Cảnh báo đỏ).")


📊 BÁO CÁO CHỈ SỐ NÂNG CAO CHO EBM
>>> REPORT FOR: OOT DATASET (Tương lai)
   • KS Statistic:       0.6156 (61.56%)
   • Brier Score:        0.046040
   • Capture Rate @ 10%: 51.52% (Tóm được bao nhiêu Bad trong top 10% rủi ro nhất)
   • Capture Rate @ 20%: 76.07%
------------------------------------------------------------
>>> STABILITY (PSI - Train vs OOT): 0.0287
   ✅ PSI < 0.1: Mô hình rất ổn định.


In [ ]:
# # ============================================================
# # 4. OPTUNA TUNING – EBM OPTIMIZED
# # ============================================================
# from optuna.pruners import HyperbandPruner
# print("\n>>> [3/6] OPTIMIZING MODEL WITH OPTUNA...")

# def objective(trial):
#     params = {
#         "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.05),
#         "interactions": trial.suggest_int("interactions", 0, 20),
#         "max_bins": trial.suggest_int("max_bins", 128, 512),
#         "outer_bags": trial.suggest_int("outer_bags", 4, 20),
#         "min_samples_leaf": trial.suggest_int("min_samples_leaf", 2, 100),
#     }

#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#     gini_scores = []

#     for train_idx, val_idx in skf.split(X_train, y_train):
#         X_tr, X_va = X_train.iloc[train_idx], X_train.iloc[val_idx]
#         y_tr, y_va = y_train.iloc[train_idx], y_train.iloc[val_idx]

#         model = ExplainableBoostingClassifier(
#             monotone_constraints=mono_constraints_list,
#             random_state=42,
#             n_jobs=-1,
#             **params
#         )

#         model.fit(X_tr, y_tr)

#         prob = model.predict_proba(X_va)[:, 1]
#         auc = roc_auc_score(y_va, prob)
#         gini_scores.append(2*auc - 1)

#     return np.mean(gini_scores)

# study = optuna.create_study(direction="maximize",
#     pruner=optuna.pruners.MedianPruner(
#         n_startup_trials=5,
#         n_warmup_steps=1,
#         interval_steps=1

#     ))
# study.optimize(objective, n_trials=50)

# print("\n>>> BEST PARAMETERS FOUND:")
# print(study.best_params)
# print(f"Best Gini (OOS): {study.best_value:.4f}")

# # Train lại model optimized
# ebm_opt = ExplainableBoostingClassifier(
#     monotone_constraints=mono_constraints_list,
#     random_state=42,
#     n_jobs=-1,
#     **study.best_params
# )
# ebm_opt.fit(X_train, y_train)

# print("\n>>> [4/6] GINI – EBM OPTIMIZED")
# gini_opt_oos = calculate_gini(ebm_opt, X_val, y_val, "OOS-Optimized")
# gini_opt_oot = calculate_gini(ebm_opt, X_test, y_test, "OOT-Optimized")

# # Visualization
# fig1 = plot_optimization_history(study)
# fig2 = plot_param_importances(study)
# fig1.show()
# fig2.show()

# # Save model
# pickle.dump(ebm_opt, open("ebm_optimized.pkl", "wb"))

In [ ]:
# import optuna.visualization as vis

# # 1. Vẽ Slice Plot cho tất cả các tham số
# fig_slice = vis.plot_slice(study)
# fig_slice.show()

# # 2. Hoặc vẽ Slice Plot cho một vài tham số quan trọng bạn muốn đưa vào bài viết
# fig_specific = vis.plot_slice(study, params=["learning_rate", "min_samples_leaf", "interactions"])
# fig_specific.show()

In [ ]:
# # Train lại model optimized
# ebm_opt = ExplainableBoostingClassifier(
#     monotone_constraints=mono_constraints_list,
#     random_state=42,
#     n_jobs=-1,
#     learning_rate=0.041459457759190334, 
#     interactions=8, 
#     max_bins=325, 
#     outer_bags=11, 
#     min_samples_leaf=27
# )
# ebm_opt.fit(X_train, y_train)

# print("\n>>> [4/6] GINI – EBM OPTIMIZED")
# gini_opt_oos = calculate_gini(ebm_opt, X_val, y_val, "OOS-Optimized")
# gini_opt_oot = calculate_gini(ebm_opt, X_test, y_test, "OOT-Optimized")


In [ ]:
# pickle.dump(ebm_opt, open("ebm_optimized.pkl", "wb"))

In [ ]:
# # ============================================================
# # 5. MODEL EXPLAINABILITY
# # ============================================================

# print("\n>>> [5/6] EXPLAINABILITY REPORT...")

# # 5.1. GLOBAL FEATURE IMPORTANCE
# global_importance = ebm_opt.explain_global()

# print("\n>>> TOP FEATURES (Global Importance):")
# for i, (name, score) in enumerate(zip(
#         global_importance.data()['names'],
#         global_importance.data()['scores'])):
#     print(f"{i+1:2d}. {name}: {score:.4f}")

# # 5.2. HIỂN THỊ BẰNG TRÌNH DUYỆT (nếu chạy trong notebook thì hiện ngay)
# try:
#     show(global_importance)
# except:
#     print("   (Không thể render giao diện trực tiếp — vẫn tiếp tục.)")

# # 5.3. LOCAL EXPLANATION (giải thích 1 khách hàng bất kỳ)
# sample_index = 0 
# sample_X = X_test.iloc[[0]]  # vẫn là DataFrame 1 hàng
# sample_y = y_test.iloc[0]    # Series scalar, 1D

# local_exp = ebm_opt.explain_local(X_test, y_test)
#   # Series



# print("\n>>> LOCAL EXPLANATION SAMPLE:")
# show(local_exp)
# print(local_exp.data(0))


# try:
#     importances = ebm_opt.term_importances()
# except:
#     try:
#         importances = ebm_opt.feature_importances_
#     except:
#         raise ValueError("Không tìm thấy thuộc tính term_importances hoặc feature_importances.")

# terms = ebm_opt.term_names_

# print("\n>>> TOP INTERACTIONS:")
# for name, imp in sorted(zip(terms, importances), key=lambda x: x[1], reverse=True):
#     if "&" in name:  # interaction luôn có ký tự &
#         print(f"{name}: {imp:.4f}")


# # Lưu phần explain vào file
# pickle.dump(global_importance, open("ebm_global_exp.pkl", "wb"))
# pickle.dump(local_exp, open("ebm_local_exp.pkl", "wb"))

Global importance: 
- Giá trị tuyệt đối trung bình của điểm số (Average Absolute Score) mà biến đó đóng góp vào dự báo trên toàn bộ tập dữ liệu: Trung bình, biến này làm thay đổi dự đoán (log-odds) bao nhiêu đơn vị? Số càng lớn = Biến càng quan trọng.

Cách tính, ví dụ với biến SOHUUNHA:
- Bước 1: Tính Score cho từng người: Với mỗi khách hàng trong tập dữ liệu, mô hình xem khách hàng đó có nhà hay không (SOHUUNHA = 1 hoặc 0). Sau đó, nó tra cứu trong bảng hàm hình dạng xem giá trị đó tương ứng với bao nhiêu điểm (ví dụ: Có nhà thì $-0.8$ điểm, không nhà thì $+1.3$ điểm rủi ro).

- Bước 2: Lấy trị tuyệt đối: Vì mô hình muốn đo lường mức độ tác động (dù là làm tăng hay giảm rủi ro đều được coi là quan trọng), nên nó lấy trị tuyệt đối của tất cả các điểm số đó: $|-0.8| = 0.8$ và $|1.3| = 1.3$.

- Bước 3: Tính trung bình: Cộng tất cả các trị tuyệt đối này lại và chia cho tổng số khách hàng. Kết quả cuối cùng chính là con số bạn thấy.

Tác động của biến SOHUUNHA: 
- SOHUUNHA ≈ 0 (Không sở hữu nhà): * Giá trị Score nằm ở mức khoảng +0.8. Những khách hàng không có nhà riêng có xu hướng làm tăng xác suất của biến mục tiêu
- SOHUUNHA ≈ 1 (Có sở hữu nhà): * Giá trị Score giảm mạnh xuống mức khoảng -1.5. Việc sở hữu nhà đóng góp một giá trị âm rất lớn vào tổng điểm. Điều này làm giảm mạnh xác suất xảy ra biến mục tiêu

In [ ]:
# # ============================================================
# # 6. FINAL REPORT – PSI, COMPARISON, SUMMARY
# # ============================================================

# print("\n>>> [6/6] FINAL MODEL REPORT...")

# # 6.1. Tính PSI của 2 biến chính giữa Train – OOT
# psi_report = {}
# for col in ["INCOME", "CBAL", "RATIO_DTI", "RATIO_UTILIZATION"]:
#     try:
#         psi_report[col] = calculate_psi(
#             X_train[col].values,
#             X_test[col].values
#         )
#     except:
#         psi_report[col] = None

# print("\n>>> PSI REPORT (Train → OOT):")
# for k,v in psi_report.items():
#     print(f"   {k}: {v:.4f}")

# # 6.2. So sánh Gini Baseline vs Optimized
# print("\n>>> GINI COMPARISON")
# print(f"Baseline – OOS: {gini_base_oos:.4f}")
# print(f"Optimized – OOS: {gini_opt_oos:.4f}")
# print(f"Baseline – OOT: {gini_base_oot:.4f}")
# print(f"Optimized – OOT: {gini_opt_oot:.4f}")

# # 6.3. Lưu bản summary ra file TXT
# # Best Hyperparameters:
# # {study.best_params}
# # """
# summary_text = f"""
# =============================
# EBM CREDIT SCORING SUMMARY
# =============================

# GINI – OOS:
#  - Baseline:  {gini_base_oos:.4f}
#  - Optimized: {gini_opt_oos:.4f}

# GINI – OOT:
#  - Baseline:  {gini_base_oot:.4f}
#  - Optimized: {gini_opt_oot:.4f}

# PSI (Train → OOT):
# {psi_report}

# """

# with open("model_summary.txt", "w") as f:
#     f.write(summary_text)

# print("\n>>> SUMMARY EXPORTED → model_summary.txt\n")


Mô hình rất ổn định khi chuyển từ TRAIN → OOT

In [ ]:
# # 6.4. Tính PSI cho Score (Xác suất dự báo) - QUAN TRỌNG NHẤT
# prob_train = ebm_opt.predict_proba(X_train)[:, 1]
# prob_oot = ebm_opt.predict_proba(X_test)[:, 1]

# psi_score = calculate_psi(prob_train, prob_oot)
# print(f"\n>>> FINAL MODEL STABILITY (PSI Score): {psi_score:.4f}")
# if psi_score < 0.1:
#     print("    => Kết luận: Mô hình cực kỳ ổn định qua thời gian.")
# elif psi_score < 0.25:
#     print("    => Kết luận: Mô hình có sự biến động nhẹ, cần giám sát.")
# else:
#     print("    => Kết luận: CẢNH BÁO! Mô hình không ổn định, cần kiểm tra lại dữ liệu OOT.")

In [ ]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# # ============================================================
# # 7. NÂNG CAO: MONOTONICITY, CONFUSION MATRIX & RATING
# # ============================================================
# print("\n>>> [7/7] ADVANCED ANALYTICS & RATING...")

# # 7.1. Kiểm tra tính đơn điệu (Monotonicity Check)
# # EBM lưu các hàm đóng góp trong explain_global. 
# # Bạn có thể vẽ lại để kiểm tra xem các biến ép đơn điệu có đúng đường thẳng/bậc thang không.
# def check_monotonicity(ebm_model, feature_name):
#     exp = ebm_model.explain_global(name=feature_name)
#     plt.figure(figsize=(8, 5))
#     plt.step(exp.data(0)['names'], exp.data(0)['scores'], where='post')
#     plt.title(f"Monotonicity Check: {feature_name}")
#     plt.xlabel(feature_name)
#     plt.ylabel("Logit Contribution")
#     plt.grid(True)
#     plt.show()

# # Chạy thử cho 2 biến tiêu biểu
# # check_monotonicity(ebm_opt, 'MAX_DPD_12M_OBS') 
# # check_monotonicity(ebm_opt, 'LTV')

# # 7.2. Ma trận nhầm lẫn tại điểm cắt (Cut-off)
# # Giả sử ngân hàng chọn Cut-off là 10% (PD > 0.1 là từ chối)
# threshold = 0.1
# y_prob_oot = ebm_opt.predict_proba(X_test)[:, 1]
# y_pred_oot = (y_prob_oot >= threshold).astype(int)

# print(f"\n>>> CONFUSION MATRIX (At Threshold {threshold}):")
# cm = confusion_matrix(y_test, y_pred_oot)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Good (0)', 'Bad (1)'])
# disp.plot(cmap='Blues')
# plt.title(f"Confusion Matrix at Cut-off {threshold}")
# plt.show()

# print(classification_report(y_test, y_pred_oot))

# # 7.3. Phân hạng khách hàng (Credit Rating)
# def assign_rating(pd):
#     if pd <= 0.01: return 'AAA (Rất an toàn)'
#     if pd <= 0.03: return 'AA'
#     if pd <= 0.05: return 'A'
#     if pd <= 0.10: return 'BBB'
#     if pd <= 0.20: return 'BB'
#     if pd <= 0.50: return 'B'
#     return 'C (Rủi ro cao - Từ chối)'

# # Tạo bảng kết quả cuối cùng cho tập Test
# df_result = pd.DataFrame({
#     'Actual': y_test.values,
#     'PD': y_prob_oot
# })

# df_result['Rating'] = df_result['PD'].apply(assign_rating)

# print("\n>>> THỐNG KÊ HẠNG KHÁCH HÀNG TRÊN TẬP OOT:")
# rating_dist = df_result['Rating'].value_counts().sort_index()
# print(rating_dist)

# # Tính tỷ lệ nợ xấu thực tế trên mỗi hạng (Để kiểm tra tính phân tách)
# bad_rate_by_rating = df_result.groupby('Rating')['Actual'].mean()
# print("\n>>> TỶ LỆ NỢ XẤU THỰC TẾ THEO HẠNG (Bad Rate by Grade):")
# print(bad_rate_by_rating)

In [ ]:
# # ============================================================
# # 8. CREDIT SCORECARD CALCULATION (300 - 850)
# # ============================================================
# print("\n>>> [8/7] CONVERTING PD TO CREDIT SCORE (300-850)...")

# def calculate_credit_score(pd, pdo=20, base_score=600, base_odds=50):
#     """
#     Chuyển đổi PD sang Credit Score dựa trên công thức chuẩn:
#     Score = Offset + Factor * ln(odds)
#     - pdo: Points to Double the Odds (thường là 20 hoặc 50)
#     - base_score: Điểm cơ sở (ví dụ 600 điểm)
#     - base_odds: Tỷ lệ odds tương ứng với điểm cơ sở (ví dụ 50:1)
#     """
#     # Tránh chia cho 0 hoặc log(0)
#     pd = np.clip(pd, 0.0001, 0.9999)
    
#     factor = pdo / np.log(2)
#     offset = base_score - factor * np.log(base_odds)
    
#     odds = (1 - pd) / pd
#     score = offset + factor * np.log(odds)
    
#     return np.round(score).astype(int)

# # Áp dụng cho tập Test (OOT)
# df_result['Credit_Score'] = calculate_credit_score(df_result['PD'])

# # Giới hạn điểm trong khoảng 300 - 850 (như chuẩn FICO)
# df_result['Credit_Score'] = df_result['Credit_Score'].clip(300, 850)

# print("\n>>> THỐNG KÊ ĐIỂM TÍN DỤNG TRÊN TẬP OOT:")
# print(df_result['Credit_Score'].describe())

# # Vẽ biểu đồ phân phối điểm số
# plt.figure(figsize=(10, 6))
# plt.hist(df_result['Credit_Score'], bins=50, color='skyblue', edgecolor='black')
# plt.axvline(df_result['Credit_Score'].mean(), color='red', linestyle='dashed', linewidth=2, label='Average Score')
# plt.title('Distribution of Credit Scores (OOT Set)')
# plt.xlabel('Credit Score')
# plt.ylabel('Number of Customers')
# plt.legend()
# plt.show()

# # Kiểm tra sự tương quan giữa Hạng và Điểm trung bình
# print("\n>>> ĐIỂM TRUNG BÌNH THEO HẠNG:")
# print(df_result.groupby('Rating')['Credit_Score'].mean().sort_values(ascending=False))